In [20]:
from dotenv import load_dotenv
import dspy

load_dotenv()

ANTHROPIC_LITELLM_PREFIX = "anthropic"
MODEL = "claude-3-5-sonnet-20241022"

lm = dspy.LM(f'{ANTHROPIC_LITELLM_PREFIX}/{MODEL}')
dspy.configure(lm=lm)

In [21]:
module = dspy.ChainOfThought('question -> answer: int')
module(question="What is 2+2?")

Prediction(
    reasoning='This is a basic addition problem. Two plus two equals four.',
    answer=4
)

In [22]:
fact_checking = dspy.ChainOfThought('claims -> verdicts: list[bool]')
fact_checking(claims=["Python was released in 1991.", "Python is a compiled language."])

Prediction(
    reasoning='Let me evaluate each claim:\n\n1. "Python was released in 1991" - This is TRUE. Python was indeed first released by Guido van Rossum in 1991.\n\n2. "Python is a compiled language" - This is FALSE. Python is primarily an interpreted language. While Python code is compiled to bytecode (.pyc files) before execution, it\'s not a compiled language in the traditional sense like C++ or Java. Python code is executed by an interpreter, not directly compiled to machine code.',
    verdicts=[True, False]
)

In [41]:
from textwrap import dedent

ques = dedent(
    """
    Which of the following is the function if the integral is 8?

    - f(x) = 4x between 0 and 1
    - f(x) = 4x between 4 and 8
    - f(x) = 4x between 0 and 2
    """
)

ans = dedent(
    """
    f(x) = 4x between 0 and 1
    """
)

class ExplainMistake(dspy.Signature):
    """
    Describe how the user would approach this problem and why they answered it incorrectly.
    """
    question = dspy.InputField(desc="The question posed to the student.")
    answer = dspy.InputField(desc="The incorrect answer that the student provided.")
    mistake = dspy.OutputField(desc="The mistake that the student made and why it would lead to a false answer.")

class PrescribeLesson(dspy.Signature):
    """
    You're a math teacher for high school students that utilizes principles of learning science to help your students understand concepts better.
    You are a diligent teacher that can walk students through any standard concept that appears in algebra, trigonometry, or calculus.

    A student has just provided you with a mistake that they made after being given a question.
    Develop a brief lesson plan (one that can be conveyed in 3 paragraphs or less) that will explain to the student concepts they need to know in order to not make this mistake again.
    """
    question = dspy.InputField(desc="The original question asked to the student")
    answer= dspy.InputField(desc="The incorrect answer that the student gave.")
    reasoning = dspy.InputField(desc="The suggested reasoning of the student.")
    mistake = dspy.InputField(desc="The mistake or incorrect answer that the user selected, along with a justification.")
    lesson = dspy.OutputField(desc="The hierarchal lesson plan that the student needs to fix this issue. Structured in bullet-point format.")
    example = dspy.OutputField(desc="Provide a step-by-step example for how you would solve the problem to the student.")


# explain the mistake
explain_mistake = dspy.ChainOfThought(ExplainMistake)
mis = explain_mistake(question=ques, answer=ans)

# now develop a lesson plan around this
lesson_plan = dspy.ChainOfThought(PrescribeLesson)
new_lesson = lesson_plan(question=ques, answer=ans, **mis.toDict())
lesson = new_lesson.lesson
lesson_example = new_lesson.example

print(lesson + "\n")
print(lesson_example)

• Understanding Definite Integrals
    - Definition of a definite integral as area under the curve
    - Steps for evaluating definite integrals
    - Importance of checking all bounds given in options

• Integration Rules
    - Power rule for integration: ∫xⁿ dx = (xⁿ⁺¹)/(n+1) + C
    - Coefficient rule: ∫(kf(x))dx = k∫f(x)dx
    - Evaluating at bounds: F(b) - F(a)

• Area Calculation
    - Understanding that the integral represents area
    - Checking if result matches the given value (8 in this case)
    - Verifying answer by testing all options

Let's solve this step by step:

1. We need to find which option gives an integral of 8
2. For f(x) = 4x, let's integrate:
   - ∫4x dx = 2x² + C

3. Now let's check each option:
   a) From 0 to 1:
      - 2(1)² - 2(0)² = 2 - 0 = 2 ✗
   
   b) From 4 to 8:
      - 2(8)² - 2(4)² = 128 - 32 = 96 ✗
   
   c) From 0 to 2:
      - 2(2)² - 2(0)² = 8 - 0 = 8 ✓

4. Therefore, f(x) = 4x between 0 and 2 is correct because it gives us 8
